In [103]:
### Importing required libraries
'''
this libraries required for current application
all these libraries contained in basic python installation except
for the pandas. pandas needed to be install via 'pip install pandas'
'''

import pandas as pd ## pandas library for working with tabular data
import numpy as np  ## numpy librari for working with arrays
import datetime     ## datetime library for working with dates
import warnings     ## warnings library, to manage warnings
warnings.filterwarnings('ignore') ## disable notofocations and warnings (sinse we are experimenting here)

In [104]:
### Import section

'''
First of all we need to load input raw datasets.
'''

### JHU raw data imports
inp_jhu_c = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
inp_jhu_d = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

### WHO raw data input
inp_who = pd.read_csv('inp/WHO-COVID-19-global-data.csv')

### Coutry grouping file with 'gavi' columns
cont_gr = pd.read_csv('inp/Country_Groupings.csv')

In [105]:
### Inspecting raw input data

In [106]:
# inp_jhu_c.head(1)

In [107]:
# inp_who.head(1)

In [108]:
# cont_gr.head(1)

In [50]:
### Searching in raw datasets

# inp_jhu_c.loc[inp_jhu_c['Country/Region']=='China']
# inp_who.loc[inp_who['Country Name']=='China'].tail(2)

In [51]:
'''
The goal is to build dataset which will contain cases/deathes from both jhu and who raw data, moreover we need to add 'gavi' columns to output dataset
As we can see raw datasets has different structure. We need to reshape jhu raw dataset, and prepare both raw dataset for merging. 
We are planning to merge data by country key and date key. Let's inspect countries from raw datasets
'''

"\nThe goal is to build dataset which will contain cases/deathes from both jhu and who raw data, moreover we need to add 'gavi' columns to output dataset\nAs we can see raw datasets has different structure. We need to reshape jhu raw dataset, and prepare both raw dataset for merging. \nWe are planning to merge data by country key and date key. Let's inspect countries from raw datasets\n"

In [52]:
### Functions 

'''
this function compares two lists and return values that are not matching in both lists
'''
def returnNotMatches(a, b):
    return [[x for x in a if x not in b], [x for x in b if x not in a]]

In [53]:
### Generating country lists from raw data for inspection.

jhu_country_list = inp_jhu_c['Country/Region'].unique().tolist() 
jhu_country_list.sort()
who_country_list = inp_who['Country'].unique().tolist() 
who_country_list.sort()
gavi_country_list = cont_gr['country'].unique().tolist() 
gavi_country_list.sort()

In [54]:
# jhu_country_list
len(jhu_country_list)

188

In [55]:
# who_country_list
len(who_country_list)

216

In [56]:
# gavi_country_list
len(gavi_country_list)

195

In [57]:
### Comparing country names

# returnNotMatches(gavi_country_list, jhu_country_list)
# returnNotMatches(gavi_country_list, who_country_list)

In [58]:
'''
we will take gavi coutry list as a reference. that means thas we have to rename countries in both jhu and who datasets.
'''

'\nwe will take gavi coutry list as a reference. that means thas we have to rename countries in both jhu and who datasets.\n'

In [59]:
countries_to_remove = ['Czechia', 'Holy See', 'Liechtenstein', 'Taiwan*', 'Kosovo','Kosovo[1]', 'MS Zaandam', 'Western Sahara', 'Diamond Princess']

In [60]:
jhu_to_rename = {"Tanzania": "Tanzania, United Republic of", "Congo (Brazzaville)": "Congo, Republic of",
                 "Congo (Kinshasa)": "Congo, Democratic Republic of the",
                 "Cote d'Ivoire": "Côte d'Ivoire", "Laos": "Lao People's Democratic Republic",
                 "Burma": "Myanmar", "Vietnam": "Viet Nam", "Syria": "Syrian Arab Republic",
                 "Eswatini": "Swaziland", "Libya": "Libyan Arab Jamahiriya",
                 "Cabo Verde": "Cape Verde", "Brunei": "Brunei Darussalam",
                 "West Bank and Gaza": "Palestinian Territory",
                 "North Macedonia": "Macedonia, Republic of", "Korea, South": "Korea, Republic of",
                 "Russia": "Russian Federation",
                 "Iran": "Iran, Islamic Republic of",
                 "United Kingdom": "United Kingdom of Great Britain & Northern Ireland",
                 "US": "United States"}

In [61]:
who_to_rename = {"United Republic of Tanzania": "Tanzania, United Republic of", "Congo": "Congo, Republic of",
                 "Democratic Republic of the Congo": "Congo, Democratic Republic of the",
                 "Côte d’Ivoire": "Côte d'Ivoire", "Laos": "Lao People's Democratic Republic",
                 "Burma": "Myanmar", "Vietnam": "Viet Nam", "Syria": "Syrian Arab Republic", "Eswatini": "Swaziland",
                 "Libya": "Libyan Arab Jamahiriya",
                 "Cabo Verde": "Cape Verde", "Brunei": "Brunei Darussalam",
                 "occupied Palestinian territory, including east Jerusalem": "Palestinian Territory",
                 "North Macedonia": "Macedonia, Republic of", "Republic of Korea": "Korea, Republic of",
                 "Russia": "Russian Federation",
                 "Iran (Islamic Republic of)": "Iran, Islamic Republic of",
                 "The United Kingdom": "United Kingdom of Great Britain & Northern Ireland",
                 "United States of America": "United States", "Bolivia (Plurinational State of)": "Bolivia",
                 "Republic of Moldova": "Moldova",
                 "Venezuela (Bolivarian Republic of)": "Venezuela", "Sint Maarten": "St Matrin", "Réunion": "Reunion"}

In [62]:
### JHU data preparation section
'''
JHU dataset needs to be transposed. Dates from column names should be trasformed into values to form column with dates.
to extract dates from column names we need to do following:
'''

jhu_columns = inp_jhu_c.iloc[:,4:].columns

'''
JHU max date
'''
jhu_max_date = inp_jhu_c.columns[-1]
x = datetime.datetime.strptime(jhu_max_date, "%m/%d/%y")
jhu_max_date = x.strftime("%m/%d/%Y")

'''
here we use indexing. It is tricky approach by pandas (similar approach used for lists managing in python). So [:,4:] means that 
we take all rows (:) and take column names from fourth one until the letest one (4:) and take only columns values from it (.columns)
Thants how we got the walues from column names
'''

'''
Remove and rename countries
'''
inp_jhu_c['Country/Region'].replace(jhu_to_rename, inplace=True)
inp_jhu_c = inp_jhu_c[~inp_jhu_c['Country/Region'].isin(countries_to_remove)]
inp_jhu_d['Country/Region'].replace(jhu_to_rename, inplace=True)
inp_jhu_d = inp_jhu_d[~inp_jhu_d['Country/Region'].isin(countries_to_remove)]


'''
Sort dataset to extract data in right order
'''
inp_jhu_c = inp_jhu_c.sort_values(['Country/Region','Province/State'])
inp_jhu_d = inp_jhu_d.sort_values(['Country/Region','Province/State'])

'''
Here we are taking only values from all rows and from 4th column untill the last one. 
We will need this walues to form reshaped jhu dataset
'''
values_c = inp_jhu_c.iloc[:,4:].values.tolist()
values_d = inp_jhu_d.iloc[:,4:].values.tolist()

'''
We've got a list of lists. Each row of values represent one element of the list 
[ [1,2,3,4...],[1,2,3,4...],[1,2,3,4...] ]
we need to make it flatten (to make one big list of values), we are using nested loop for that:
'''

flat_list_c = []
for sublist in values_c:
    for item in sublist:
        flat_list_c.append(item)
flat_list_d = []
for sublist in values_d:
    for item in sublist:
        flat_list_d.append(item)
        
'''
Creating the variaty list on countries/regions/province/states just in case we will need them.
Some of them will be used, some of them no.
'''
### JHU_regions/countries etc lists
jhu_regions_list_nn = inp_jhu_c['Province/State'].to_list()
jhu_regions = inp_jhu_c[inp_jhu_c['Province/State'].notnull()]
jhu_regions_list = jhu_regions['Province/State'].to_list()
jhu_countries_list = inp_jhu_c['Country/Region'].unique().tolist() 
jhu_countries_list_2 = inp_jhu_c['Country/Region'].tolist() 
jhu_count_regions = jhu_regions_list+jhu_countries_list


In [63]:
### JHU reshaping process
'''
Creating df with countries and provinces as a part of out jhu reshaping process
'''

'''
New reshaped jhu dataset: 
1 column - dates
2 colunm - 'Province/State'
3 column - 'Country/Region'
'''
df1 = pd.DataFrame({'date': jhu_columns})  ### this is build in pandas function for DataFrame(dataset) creation
df2 = inp_jhu_c[['Province/State','Country/Region']]


'''
We need to get resulting dataset as a table of time perions form 01/22/20 untill now and each country should have this time period. 
To do that we need to use a trick: to add same zero key to each dataframe (df1 and df2) and to merge them in 'outer' way by that key
'''
df1['key'] = 0
df2['key'] = 0
jhu_resh = df1.merge(df2, how='outer')

'''
Newx step is adding 'cases' and 'deaths' to our new df
'''

jhu_resh = jhu_resh.sort_values(['Country/Region', 'Province/State']) #just in case (we did it earlier)

'''
new 'confirmed' column - values from flatten_c list
new 'deaths' column - values from flatten_d list
'''
jhu_resh['confirmed'] = flat_list_c
jhu_resh['deaths'] = flat_list_d
jhu_resh['date'] = pd.to_datetime(jhu_resh['date']).dt.strftime('%m/%d/%Y')  # we need to make this dates real dates to have an ability to compare and work with them

'''
again taking country and province lists to check if everithing is good and we did not miss any cointry or province
'''
res_list = jhu_resh['Country/Region'].unique().tolist()
res_list_r = jhu_resh['Province/State'].unique().tolist()
jhu_resh_c_r_list = res_list+res_list_r

res_only_regions = jhu_resh.dropna(subset=["Province/State"])  # df contains ONLY countries
res_only_countries = jhu_resh[jhu_resh['Province/State'].isna()] # df contains ONLY province

In [64]:
# jhu_resh

In [65]:
inp_who

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-02-24T00:00:00Z,AF,Afghanistan,EMRO,1,1,0,0
1,2020-02-25T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
2,2020-02-26T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
3,2020-02-27T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
4,2020-02-28T00:00:00Z,AF,Afghanistan,EMRO,0,1,0,0
...,...,...,...,...,...,...,...,...
20425,2020-06-04T00:00:00Z,ZW,Zimbabwe,AFRO,16,222,0,4
20426,2020-06-05T00:00:00Z,ZW,Zimbabwe,AFRO,0,222,0,4
20427,2020-06-06T00:00:00Z,ZW,Zimbabwe,AFRO,43,265,0,4
20428,2020-06-07T00:00:00Z,ZW,Zimbabwe,AFRO,14,279,0,4


In [66]:
'''
JHU dataset preparation and reshaping is DONE
now it is time to WHO dataset preparation
'''

### WHO dataprep
who = inp_who[~inp_who['Country'].isin(countries_to_remove)] ### removing countries that we are not using
who['Country'].replace(who_to_rename, inplace=True) ### rename countries to match gavi naming
who = who[['Date_reported','Country','Cumulative_deaths','Cumulative_cases']] ### keep only necessary columns
who['day'] = pd.to_datetime(who.Date_reported).dt.strftime('%m/%d/%Y') # we need to make this dates real dates to have an ability to compare and work with them
who = who[who.Date_reported >='01/22/2020'] ### keeping information only for time perion after 01/22/2020

who_countries_list = who['Country'].unique().tolist() ### generationg country list to make sure that all countries are in place

'''WHO max date'''
who_max_date = who['Date_reported'].max()

who['Date_reported'] = pd.to_datetime(who['Date_reported']).dt.strftime('%m/%d/%Y')


In [67]:
who

,Date_reported,Country,Cumulative_deaths,Cumulative_cases,day
0,02/24/2020,Afghanistan,0,1,02/24/2020
1,02/25/2020,Afghanistan,0,1,02/25/2020
2,02/26/2020,Afghanistan,0,1,02/26/2020
3,02/27/2020,Afghanistan,0,1,02/27/2020
4,02/28/2020,Afghanistan,0,1,02/28/2020
...,...,...,...,...,...
20425,06/04/2020,Zimbabwe,4,222,06/04/2020
20426,06/05/2020,Zimbabwe,4,222,06/05/2020
20427,06/06/2020,Zimbabwe,4,265,06/06/2020
20428,06/07/2020,Zimbabwe,4,279,06/07/2020


In [68]:
'''
Next step is to deal with CHINA and AUSTRALIA. In JHU dataset they are not tracking them as a whole countries. They splitted them by province/states. 
So we need to aggregate
'''

######## CHINA

### extraction CHINA from JHU reshaped df
china = jhu_resh.loc[jhu_resh['Country/Region']=='China']
china = china.groupby(['Country/Region','date'],as_index=False)['confirmed','deaths'].sum() ### summing 'confirmed' and 'deaths' by CHINA
china['Province/State']=np.nan ### creating 'Province/State' and fill it up with NaN values

### extraction CHINA from WHO df
who_china = who.loc[who['Country']=='China']

china_merged = pd.merge(china,who_china,left_on=['Country/Region','date'],right_on=['Country','Date_reported'],how='outer') # merging jhu china with who china (outer)
china_merged = china_merged[['date','Province/State','Country/Region','confirmed','deaths','Cumulative_deaths','Cumulative_cases']] # keep only needed colomns
china_merged = china_merged.rename(columns={'confirmed':'jhu_cases','deaths':'jhu_deaths','Cumulative_deaths':'WHO_deaths','Cumulative_cases':'WHO_cases'}) # rename columns
'''
we did 'outer' merge because we need to have values from all dates (from 01/22/20). In who dataset min date for countries is varies, so values that are missing 
in who was replased with NaN (same as missing values). In next step we are fillin nan values with zeroes and make this values integer.
'''
china_merged[['jhu_cases', 'jhu_deaths','WHO_deaths', 'WHO_cases']] = china_merged[['jhu_cases', 'jhu_deaths','WHO_deaths', 'WHO_cases']].fillna(0).astype(int) 

######## AUSTRALIA

australia = jhu_resh.loc[jhu_resh['Country/Region']=='Australia']
australia = australia.groupby(['Country/Region','date'],as_index=False)['confirmed','deaths'].sum()
australia['Province/State']=np.nan

who_australia = who.loc[who['Country']=='Australia']

australia_merged = pd.merge(australia,who_australia,left_on=['Country/Region','date'],right_on=['Country','Date_reported'],how='outer')
australia_merged = australia_merged[['date','Province/State','Country/Region','confirmed','deaths','Cumulative_deaths','Cumulative_cases']]
australia_merged = australia_merged.rename(columns={'confirmed':'jhu_cases','deaths':'jhu_deaths','Cumulative_deaths':'WHO_deaths','Cumulative_cases':'WHO_cases'})
australia_merged[['jhu_cases', 'jhu_deaths','WHO_deaths', 'WHO_cases']] = australia_merged[['jhu_cases', 'jhu_deaths','WHO_deaths', 'WHO_cases']].fillna(0).astype(int)

### Append
'''
Just append China and Australia
'''
countries_to_add = china_merged.append(australia_merged)

In [69]:
### MAIN MERGING


merge_countries = pd.merge(res_only_countries,who,left_on=['date','Country/Region'],right_on=['Date_reported','Country'],how='outer')
merge_countries[['Cumulative_deaths','Cumulative_cases']] = merge_countries[['Cumulative_deaths','Cumulative_cases']].fillna(0)
merge_countries = merge_countries[['date','Province/State','Country/Region','confirmed','deaths','Cumulative_deaths','Cumulative_cases']]
merge_countries = merge_countries.dropna(subset=['date']) ### removing side dates from outer join
merge_countries = merge_countries.rename(columns={"confirmed":"jhu_cases","deaths":"jhu_deaths","Cumulative_cases":"WHO_cases","Cumulative_deaths":"WHO_deaths"})


merge_regions = pd.merge(res_only_regions,who,left_on=['date','Province/State'],right_on=['Date_reported','Country'],how='outer')
merge_regions[['Cumulative_deaths','Cumulative_cases']] = merge_regions[['Cumulative_deaths','Cumulative_cases']].fillna(0)
merge_regions = merge_regions[['date','Province/State','Country/Region','confirmed','deaths','Cumulative_deaths','Cumulative_cases']]
merge_regions = merge_regions.dropna(subset=['date']) ### removing side dates from outer join
merge_regions = merge_regions.rename(columns={"confirmed":"jhu_cases","deaths":"jhu_deaths","Cumulative_cases":"WHO_cases","Cumulative_deaths":"WHO_deaths"})

final_merge = merge_countries.append(merge_regions) ### append countries with provinces
final_merge = final_merge.append(countries_to_add) ### append CHINA and AUSTRALIA

'''
Adding gavi columns. Sinse we've renamed all countries based on gavi country naming we just need to merge countryregion gavi's tadaset 
'''
final_merge_f = pd.merge(final_merge,cont_gr,left_on='Country/Region',right_on='country')
# final_merge_f[['jhu_cases', 'jhu_deaths','WHO_deaths', 'WHO_cases']] = final_merge_f[['jhu_cases', 'jhu_deaths','WHO_deaths', 'WHO_cases']].astype(int) # this columns to integer

In [70]:
final_merge_f

,date,Province/State,Country/Region,jhu_cases,jhu_deaths,WHO_deaths,WHO_cases,Finance Country,cofinance_2017,cofinance_2018,...,lang,pef_type,regional_je,regional_mena,regional_yfv,wb_long_2017,wb_long_2018,wb_short_2017,wb_short_2018,who_region
0,01/22/2020,NaN,Afghanistan,0.0,0.0,0.0,0.0,Afghanistan,Initial self-financing,Initial self-financing,...,eng,1,0,0,0,LIC,LIC,LIC,LIC,EMRO
1,01/23/2020,NaN,Afghanistan,0.0,0.0,0.0,0.0,Afghanistan,Initial self-financing,Initial self-financing,...,eng,1,0,0,0,LIC,LIC,LIC,LIC,EMRO
2,01/24/2020,NaN,Afghanistan,0.0,0.0,0.0,0.0,Afghanistan,Initial self-financing,Initial self-financing,...,eng,1,0,0,0,LIC,LIC,LIC,LIC,EMRO
3,01/25/2020,NaN,Afghanistan,0.0,0.0,0.0,0.0,Afghanistan,Initial self-financing,Initial self-financing,...,eng,1,0,0,0,LIC,LIC,LIC,LIC,EMRO
4,01/26/2020,NaN,Afghanistan,0.0,0.0,0.0,0.0,Afghanistan,Initial self-financing,Initial self-financing,...,eng,1,0,0,0,LIC,LIC,LIC,LIC,EMRO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36135,06/04/2020,NaN,China,84171.0,4638.0,4645.0,84603.0,China,Fully self-financing,Non-Gavi,...,eng,Not PEF,0,0,0,UMIC,UMIC,MIC,MIC,WPRO
36136,06/05/2020,NaN,China,84177.0,4638.0,4645.0,84614.0,China,Fully self-financing,Non-Gavi,...,eng,Not PEF,0,0,0,UMIC,UMIC,MIC,MIC,WPRO
36137,06/06/2020,NaN,China,84186.0,4638.0,4645.0,84620.0,China,Fully self-financing,Non-Gavi,...,eng,Not PEF,0,0,0,UMIC,UMIC,MIC,MIC,WPRO
36138,06/07/2020,NaN,China,84191.0,4638.0,4645.0,84629.0,China,Fully self-financing,Non-Gavi,...,eng,Not PEF,0,0,0,UMIC,UMIC,MIC,MIC,WPRO


In [71]:
'''
So basicly everithing is done. JHU and WHO data combined and gavi's columns added. Next step will be related to improving requests
'''

"\nSo basicly everithing is done. JHU and WHO data combined and gavi's columns added. Next step will be related to improving requests\n"

In [72]:
### Additional columns

'''
Here we are adding additional columns to out dataset. One column with % gtowth and one column with numerical difference, both for 'cases' and 'deaths'
'''


'''
.pct_change() - returns % changing for the specific column
.round(4) - round values to 4 digits after the ,
.replace([np.inf, -np.inf], np.nan) - when .pct_change() calculates diffetence beetween 0 and positive number it returns infinity or -infinity.
                                    this function replase infinities with nan
.fillna(0) - replacing nan with 0
.clip(lower=0) - assigning lower boundary for the values to avoid negative values. (tricky part. actually % change can be negative as well as difference. but we have cases
                where we have big number for 08/05/20 and 0 for 22/01/20. In that case % change wll show negative growth. We want to avoid this. Since 
                we are dealing with cumulative ceses next values could be only bigger or equal to prevoius ones.) # in latest version replaced by mask approach
.diff() - returns numerical difference for specific column
'''
### For jhu_cases
final_merge_f['growth_jhu_cases'] = final_merge_f['jhu_cases'].pct_change().round(4).replace([np.inf, -np.inf], np.nan).fillna(0)#.clip(lower=0)
final_merge_f['diff_jhu_cases'] = final_merge_f['jhu_cases'].diff().fillna(0).astype(int)#.clip(lower=0)

### For jhu_deaths
final_merge_f['growth_jhu_deaths'] = final_merge_f['jhu_deaths'].pct_change().round(4).replace([np.inf, -np.inf], np.nan).fillna(0)#.clip(lower=0)
final_merge_f['diff_jhu_deaths'] = final_merge_f['jhu_deaths'].diff().fillna(0).astype(int)#.clip(lower=0)

### For who cases
final_merge_f['growth_who_cases'] = final_merge_f['WHO_cases'].pct_change().round(4).replace([np.inf, -np.inf], np.nan).fillna(0)#.clip(lower=0)
final_merge_f['diff_who_cases'] = final_merge_f['WHO_cases'].diff().fillna(0).astype(int)#.clip(lower=0)

### For who deaths
final_merge_f['growth_who_deaths'] = final_merge_f['WHO_deaths'].pct_change().round(4).replace([np.inf, -np.inf], np.nan).fillna(0)#.clip(lower=0)
final_merge_f['diff_who_deaths'] = final_merge_f['WHO_deaths'].diff().fillna(0).astype(int)#.clip(lower=0)


'''
Allows keep negative values where they really can be. Insted of .clip()
'''
additional_cols = ['growth_jhu_cases','diff_jhu_cases','growth_jhu_deaths',
                   'diff_jhu_deaths','growth_who_cases','diff_who_cases',
                   'growth_who_deaths','diff_who_deaths']
mask = (final_merge_f['date'] == final_merge_f['date'].min())
final_merge_f[additional_cols] = final_merge_f[additional_cols].where(~mask, other=0)

In [73]:
'''
Defining order of columns
'''

final_merge_f = final_merge_f[['date', 'Province/State', 'Country/Region',

                 'jhu_cases', 'growth_jhu_cases', 'diff_jhu_cases', 'jhu_deaths', 'growth_jhu_deaths','diff_jhu_deaths',
                 'WHO_cases', 'growth_who_cases', 'diff_who_cases', 'WHO_deaths', 'growth_who_deaths','diff_who_deaths',

                 'Finance Country', 'cofinance_2017', 'cofinance_2018', 'cofinance_2019', 'continental_africa','country',
                 'dov96', 'fragility_2017', 'fragility_2018', 'fragility_2019', 'francophone', 'gavi55', 'gavi68','gavi72',
                 'gavi73', 'gavi77', 'gavi_region', 'gavi_region_sf', 'gavi_region_short', 'global', 'indo_pacific','iso3',
                 'iso3_num', 'lang', 'pef_type', 'regional_je', 'regional_mena', 'regional_yfv', 'wb_long_2017','wb_long_2018',
                 'wb_short_2017', 'wb_short_2018', 'who_region']]

In [74]:
'''
Final request was to rename columns, to remove rows for the provinces/states, to remove column provinces/states and to make PEF Tier column integer
'''

final_merge_f_f = final_merge_f.rename(columns={'Country/region': 'Country',
                     'jhu_cases': 'JHU Cases', 'growth_jhu_cases': 'JHU Cases Growth',
                     'diff_jhu_cases': 'JHU Cases Difference',
                     'jhu_deaths': 'JHU Deaths', 'growth_jhu_deaths': 'JHU Deaths Growth',
                     'diff_jhu_deaths': 'JHU Deaths Difference',
                     'WHO_cases': 'WHO Cases', 'growth_who_cases': 'WHO Cases Growth',
                     'diff_who_cases': 'WHO Cases Difference',
                     'WHO_deaths': 'WHO Deaths', 'growth_who_deaths': 'WHO Deaths Growth',
                     'diff_who_deaths': 'WHO Deaths Difference',
                     'cofinance_2019': 'Cofinance 2019', 'fragility_2017': '2017 Fragility Status',
                     'fragility_2018': '2018 Fragility Status',
                     'fragility_2019': '2019 Fragility Status', 'gavi_region': 'Gavi Region', 'pef_type': 'PEF Tier',
                     'who_region': 'WHO Region'})
final_merge_f_f = final_merge_f_f[~final_merge_f_f['Province/State'].notna()]  # removing rows where province/stare NOT nan
final_merge_f_f = final_merge_f_f.drop(['Province/State', 'country', 'gavi_region_sf'], axis=1)
final_merge_f_f['PEF Tier'] = final_merge_f_f['PEF Tier'].replace({"Not PEF": 0}).astype(int)

In [75]:
'''
Saving output to desired format:
'''
# dates_ok = True #jhu_max_date == who_max_date

# if not dates_ok:
#     print('WHO and JHU data sets have different last dates.')
#     print('WHO max date is {} , and JHU max date is {}'.format(who_max_date, jhu_max_date))
#     print('Please insure that you are using the WHO dataset with {} as latest date'.format(jhu_max_date))
# else:
final_merge_f_f.to_excel('out_01.xlsx',index=False)
final_merge_f_f.to_csv('out_01.csv',index=False)

In [1]:
pwd

'/Users/dmitry/python_projects/pandas/WHO and GAVI/DEMO'